In [100]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls

In [101]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/AA_liver"#proportion_pxt_neh/"
time=31
pw=0.046

In [102]:
data=pd.read_csv(path+"__CPSM_MOUSE.csv")
data=data[data.exp_Time==time]
data=data[data.exp_Time==time].reset_index(drop="True")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/AA_liver__CPSM_MOUSE.csv'

In [103]:
res=[]
for pep_index in range(data.shape[0]):
    exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
    b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]])#exp_t
    
#     if (data.loc[pep_index,"Peptied"]!="IEFEGQSVDFVDPNK"):
#         continue
    
    A=[]
    for i in range(6):
        A.append(([0]*i)+exp_0[:6-i])
    A=np.array(A).transpose()
#     x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    x,residual=nnls(A,b)
    
#     ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
#     residual=sum([ abs(b[i]-y) for index,y in enumerate(ypred)])
    
    ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
    sigma=sum([ (b[i]-y)**2 for index,y in enumerate(ypred)])/(len(ypred)-1)
    cov=((sigma)*np.linalg.inv(np.matmul(A.transpose(),A)))
    var=[cov[x][x] for x in range(6)]
    
    print("\n",data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],"\n",np.corrcoef(x,exp_0)[1][0])
    
    neh= (x[2]/x[1])*2* ((1-pw)/pw) +1
    xm=sum([(index)*val / sum(x) for index,val in enumerate(x)])
    res.append([data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh,xm, 
               data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"],residual])
    
    r=x[1]/x[2]
    c=data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"]  #0.0610440961749121
    t_pw= (r*c-2)/ (r-2)
    print(pw,neh,pw*neh,"\t",c,"\t",t_pw,c/t_pw)
#     print(data.loc[pep_index,"Peptied"],x,end="\n\n")


 SMEYcIIEVNAR 2 
 0.9312473350992816
0.046 26.974985831191702 1.2408493482348182 	 1.0999993607401855 	 0.6039043043075771 1.821479583592999

 IEFEGQSVDFVDPNK 2 
 0.9131510898445198
0.046 38.1634308512112 1.7555178191557153 	 1.3499992154538638 	 0.5580524468421995 2.419126057224516

 TAVDSGIALLTNFQVTK 2 
 0.9873351958368588
0.046 26.019642280964234 1.1969035449243548 	 1.1699994653463377 	 0.1763505832121446 6.634508625008983

 EPLFGISTGNIITGLAAGAK 3 
 0.9073094667014923
0.046 37.49390651273001 1.7247196995855802 	 1.535999178886413 	 0.2944262216826155 5.216923853141666

 PSYVLSGSAMNVVFSEDEMK 2 
 0.9463768981433565
0.046 27.60947024508426 1.2700356312738759 	 1.535999178886413 	 -0.8936138695675405 -1.7188622862688459

 TAVDSGIALLTNFQVTK 3 
 0.9855116606113522
0.046 27.295243493221 1.255581200688166 	 1.1499993316829211 	 0.4401021591564476 2.6130281521161978

 AFAISGPFNVQFLVK 2 
 0.9938683495516929
0.046 21.254327672062576 0.9776990729148785 	 1.0499993897974498 	 3.138898427168556

In [104]:
sum([(index)* (val / sum(x)) for index,val in enumerate(x)])


0.813015501375263

In [105]:
res=pd.DataFrame(res)
res.columns=["Peptide","Charge","True NEH","e NEH","sum_xi","pxt_neh","residual"]
res["dif"]=abs(res["True NEH"]- res["e NEH"])
# res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]
res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]

res=res.sort_values(by="dif")
print(res.shape)
res[["Peptide","Charge","True NEH","e NEH","dif"]].head(50)

(37, 9)


,Peptide,Charge,True NEH,e NEH,dif
9,FDEHGIMVLGcGPY,2,23,23.018738,0.018738
14,QAFITAQNHGY,2,24,23.960527,0.039473
10,TSAcFEPSLDYMVTK,2,23,23.108664,0.108664
27,QNLIAEVSTK,2,19,19.130300,0.130300
19,VSQEHPVVLTK,3,19,18.695595,0.304405
20,SAYALGGLGSGIcPNK,3,30,29.191176,0.808824
2,TAVDSGIALLTNFQVTK,2,25,26.019642,1.019642
26,DGSIDLVINLPNNNTK,2,23,24.085994,1.085994
30,VAQAPWK,2,15,13.798288,1.201712
22,FLGVAEQLHNEGFK,3,27,28.213227,1.213227


In [106]:
res[["Peptide","Charge","True NEH","e NEH","dif"]].tail(30)

,Peptide,Charge,True NEH,e NEH,dif
26,DGSIDLVINLPNNNTK,2,23,24.085994,1.085994
30,VAQAPWK,2,15,13.798288,1.201712
22,FLGVAEQLHNEGFK,3,27,28.213227,1.213227
28,cEMASTGEVAcFGEGIHTAFLK,3,43,44.287557,1.287557
3,EPLFGISTGNIITGLAAGAK,3,36,37.493907,1.493907
31,GQNQPVLNITNR,2,24,22.383799,1.616201
24,FLGVAEQLHNEGFK,2,27,25.223080,1.776920
18,IMGTSPLQIDR,2,20,18.136304,1.863696
0,SMEYcIIEVNAR,2,25,26.974986,1.974986
25,IAPSFAVESMEDALK,2,33,31.018782,1.981218


In [107]:
res.to_csv("testx2x1.csv",index=False)

In [78]:
# temp=pd.read_csv("C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/pw_known_search_neh/__CPSM_MOUSE.csv")
# temp["dif"]=temp.NEH-temp.e_NEH
# temp.sort_values(by="Peptied") 